In [1]:
import os
import numpy as np
import pandas as pd
from lsst.sims.maf.db import ResultsDb
from IPython.display import display

/Users/lynnej/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
newruns = ['minion_1016',
           'astro_lsst_01_1016',
           'astro_lsst_01_1015',
           'astro_lsst_01_1017']
oldruns = ['enigma_1189', 'enigma_1281', 'enigma_1282', 'minion_1016']

In [3]:
pha_new = {}
neo_new = {}
for r in newruns:
    pha_new[r] = ResultsDb(os.path.join('.', r, 'pha_newsky'))
    neo_new[r] = ResultsDb(os.path.join('.', r, 'neo_newsky'))

In [4]:
pha_old = {}
neo_old = {}
for r in oldruns:
    pha_old[r] = ResultsDb(os.path.join('.', r, 'pha'))
    neo_old[r] = ResultsDb(os.path.join('.', r, 'neo'))

In [5]:
def make_dataframe(runs, metricName='CumulativeCompleteness'):
    summary = []
    for r in runs:
        mId = runs[r].getMetricId(metricName)
        s = runs[r].getSummaryStats(mId, summaryName=None)
        summary.append(pd.DataFrame(zip(s['metricMetadata'], s['summaryValue']*100.0), columns=['metadata', r]))
    df = summary[0]
    for i in range(1, len(summary)):
        df = pd.merge(df, summary[i], how='outer', on='metadata')
    return df

In [6]:
pha_new_df = make_dataframe(pha_new)
pha_old_df = make_dataframe(pha_old)
neo_new_df = make_dataframe(neo_new)
neo_old_df = make_dataframe(neo_old)

In [7]:
def show_match(metadata, df):
    display(df[df.metadata.str.contains(metadata)])

In [8]:
def show_diff(metadata1, metadata2, df):
    runnames = list(df.columns.values)
    runnames.remove('metadata')
    d = (df[df.metadata.str.contains(metadata2)][runnames].as_matrix() - 
         df[df.metadata.str.contains(metadata1)][runnames].as_matrix())
    diff = pd.DataFrame(d, columns=runnames)
    diff['average'] = np.nanmean(d)
    display(diff)

In [9]:
df = pha_new_df
match1 = '3 pairs in 15 nights yr 10'
show_match(match1, df)

,metadata,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016
7,PHA 3 pairs in 15 nights yr 10,73.729326,73.612485,76.205749,74.829865


In [10]:
match2 = '3 pairs in 15 nights yr 12'
show_match(match2, df)
show_diff(match1, match2, df)

,metadata,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016
141,PHA 3 pairs in 15 nights yr 12,NaN,77.530612,79.866959,79.130073


,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016,average
0,NaN,3.918127,3.66121,4.300208,3.959848


In [11]:
match3 = '3 pairs in 30 nights yr 12'
show_match(match3, df)
show_diff(match2, match3, df)

,metadata,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016
159,PHA 3 pairs in 30 nights yr 12,NaN,80.947165,82.452829,80.837906


,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016,average
0,NaN,3.416553,2.58587,1.707833,2.570085


In [12]:
match4 = '3 pairs in 30 nights trailing loss yr 12'
show_match(match4, df)
show_diff(match3, match4, df)

,metadata,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016
153,PHA 3 pairs in 30 nights trailing loss yr 12,NaN,82.867323,83.910607,81.483143


,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016,average
0,NaN,1.920158,1.457778,0.645237,1.341058


In [13]:
match5 = '3 pairs in 30 nights trailing loss, SNR=4 yr 12'
show_match(match5, df)
show_diff(match4, match5, df)

,metadata,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016
156,"PHA 3 pairs in 30 nights trailing loss, SNR=4 ...",NaN,85.634775,85.977228,83.758433


,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016,average
0,NaN,2.767453,2.066621,2.27529,2.369788


In [14]:
df = neo_new_df
match1 = '3 pairs in 15 nights yr 10'
show_match(match1, df)
match2 = '3 pairs in 15 nights yr 12'
show_match(match2, df)
show_diff(match1, match2, df)
match3 = '3 pairs in 30 nights yr 12'
show_match(match3, df)
show_diff(match2, match3, df)
match4 = '3 pairs in 30 nights trailing loss yr 12'
show_match(match4, df)
show_diff(match3, match4, df)
match5 = '3 pairs in 30 nights trailing loss, SNR=4 yr 12'
show_match(match5, df)
show_diff(match4, match5, df)

,metadata,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016
17,NEO 3 pairs in 15 nights yr 10,60.698645,57.718595,62.42445,56.811208
18,NEO 3 pairs in 15 nights yr 10,60.836294,57.718595,62.42445,56.811208


,metadata,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016
19,NEO 3 pairs in 15 nights yr 12,60.836294,61.361651,65.922897,60.41197


,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016,average
0,0.137649,3.643056,3.498447,3.600762,2.702772
1,0.000000,3.643056,3.498447,3.600762,2.702772


,metadata,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016
109,NEO 3 pairs in 30 nights yr 12,63.38195,65.518469,68.662914,63.139089


,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016,average
0,2.545656,4.156818,2.740017,2.727119,3.042403


,metadata,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016
79,NEO 3 pairs in 30 nights trailing loss yr 12,64.172126,67.446558,70.357363,63.883606


,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016,average
0,0.790176,1.928089,1.694449,0.744518,1.289308


,metadata,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016
94,"NEO 3 pairs in 30 nights trailing loss, SNR=4 ...",67.537108,71.502199,73.616514,67.030553


,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016,average
0,3.364982,4.055641,3.259152,3.146947,3.45668


In [15]:
df = pha_old_df
match2 = '3 pairs in 15 nights yr 10'
show_match(match2, df)
#show_diff(match1, match2, df)
match3 = '3 pairs in 30 nights yr 10'
show_match(match3, df)
show_diff(match2, match3, df)
match4 = '3 pairs in 30 nights trailing loss yr 10'
show_match(match4, df)
show_diff(match3, match4, df)
match5 = '3 pairs in 30 nights trailing loss, SNR=4 yr 10'
show_match(match5, df)
show_diff(match4, match5, df)

,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
7,PHA 3 pairs in 15 nights yr 10,72.286809,70.064613,74.964997,67.029049


,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
43,PHA 3 pairs in 30 nights yr 10,74.810999,73.124718,77.59089,71.435025


,minion_1016,enigma_1281,enigma_1189,enigma_1282,average
0,2.52419,3.060105,2.625893,4.405976,3.154041


,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
31,PHA 3 pairs in 30 nights trailing loss yr 10,75.24956,73.645192,78.012546,72.167027


,minion_1016,enigma_1281,enigma_1189,enigma_1282,average
0,0.438562,0.520475,0.421656,0.732002,0.528173


,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
37,"PHA 3 pairs in 30 nights trailing loss, SNR=4 ...",77.961248,76.531139,80.638334,74.686238


,minion_1016,enigma_1281,enigma_1189,enigma_1282,average
0,2.711687,2.885947,2.625788,2.519211,2.685658


In [16]:
df = neo_old_df
match2 = '3 pairs in 15 nights yr 10'
show_match(match2, df)
#show_diff(match1, match2, df)
match3 = '3 pairs in 30 nights yr 10'
show_match(match3, df)
show_diff(match2, match3, df)
match4 = '3 pairs in 30 nights trailing loss yr 10'
show_match(match4, df)
show_diff(match3, match4, df)
match5 = '3 pairs in 30 nights trailing loss, SNR=4 yr 10'
show_match(match5, df)
show_diff(match4, match5, df)

,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
7,NEO 3 pairs in 15 nights yr 10,59.461635,NaN,64.833056,NaN


,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
43,NEO 3 pairs in 30 nights yr 10,62.424463,NaN,68.361687,NaN


,minion_1016,enigma_1281,enigma_1189,enigma_1282,average
0,2.96283,NaN,3.52863,NaN,NaN


,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
31,NEO 3 pairs in 30 nights trailing loss yr 10,63.049837,NaN,68.633961,NaN


,minion_1016,enigma_1281,enigma_1189,enigma_1282,average
0,0.625375,NaN,0.272274,NaN,NaN


,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
37,"NEO 3 pairs in 30 nights trailing loss, SNR=4 ...",66.892363,NaN,71.976552,NaN


,minion_1016,enigma_1281,enigma_1189,enigma_1282,average
0,3.84253,NaN,3.34259,NaN,NaN


In [17]:
df = neo_old_df
match1 = '3 pairs in 15 nights yr 10'
show_match(match1, df)
match2 = '3 pairs in 30 nights yr 10'
show_match(match2, df)
match3 = '3 triplets in 30 nights yr 10'
show_match(match3, df)
match4 = '3 quads in 30 nights yr 10'
show_match(match4, df)
match5 = '3 pairs in 30 nights trailing loss yr 10'
show_match(match5, df)
match6 = '3 pairs in 30 nights trailing loss, SNR=4 yr 10'
show_match(match6, df)

,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
7,NEO 3 pairs in 15 nights yr 10,59.461635,NaN,64.833056,NaN


,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
43,NEO 3 pairs in 30 nights yr 10,62.424463,NaN,68.361687,NaN


,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
79,NEO 3 triplets in 30 nights yr 10,39.739229,NaN,44.497952,NaN


,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
61,NEO 3 quads in 30 nights yr 10,27.15548,NaN,31.261804,NaN


,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
31,NEO 3 pairs in 30 nights trailing loss yr 10,63.049837,NaN,68.633961,NaN


,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
37,"NEO 3 pairs in 30 nights trailing loss, SNR=4 ...",66.892363,NaN,71.976552,NaN
